In [1]:
import cv2
import numpy as np
import os

# Load the video (corrected path)
video_path = 'n4.mp4'  
if not os.path.exists(video_path):
    print("Error: Video file not found!")
    exit()

video = cv2.VideoCapture(video_path)
if not video.isOpened():
    print("Error: Unable to open video file")
    exit()

# Read the first frame
ret, prev_frame = video.read()
if not ret:
    print("Retrying to read the first frame...")
    ret, prev_frame = video.read()
   
if not ret:
    print("Error reading video file")
    exit()

# Convert first frame to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Lucas-Kanade Optical Flow parameters
lk_params = dict(winSize=(35, 35),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Detect good features to track
prev_corners = cv2.goodFeaturesToTrack(prev_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Create a mask for drawing motion vectors
mask = np.zeros_like(prev_frame)

while True:
    ret, curr_frame = video.read()
    if not ret:
        break

    # Convert current frame to grayscale
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Compute Optical Flow
    curr_corners, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)

    if curr_corners is not None and status is not None:
        # Filter valid points
        good_old = prev_corners[status == 1]
        good_new = curr_corners[status == 1]

        for (old, new) in zip(good_old, good_new):
            x_old, y_old = old.ravel()
            x_new, y_new = new.ravel()
            mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 0, 255), 2)
            curr_frame = cv2.circle(curr_frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)

        # Combine mask with the frame
        output = cv2.add(curr_frame, mask)

        # Show the frame in a new window
        cv2.imshow("Optical Flow", output)

        # Update previous frame and corners
        prev_corners = good_new.reshape(-1, 1, 2)

    prev_gray = curr_gray.copy()

    # Break if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()


2025-03-20 22:16:17.098 Python[2641:97796] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
